# Testing crawler and site structure

In [4]:
import requests
import re

import urllib.parse as urlparse

from bs4 import BeautifulSoup as bs

URL_MAIN = 'https://www.inform.kz'
URL_ARCHIVE = 'https://www.inform.kz/ru/archive'

session = requests.Session()


## Check for the crawling links

### Check for article links in the first page of archives

In [3]:
url_links = 'https://www.inform.kz/ru/archive?date=01.01.2012'
response = session.get(url_links)
soup = bs(response.content, 'html.parser')

links = soup.find('div', class_='news-list__col').find_all('a')
links = set([urlparse.urlparse(URL_MAIN + link['href'].strip()).geturl() for link in links])
links = list(links)
print(f'Links for articles (total {len(links)} links) from main page:\n{links[:5]}...')

Links for articles (total 20 links) from main page:
['https://www.inform.kz/ru/sud-evraziyskogo-ekonomicheskogo-soobschestva-nachnet-svoyu-rabotu-v-minske_a2430379', 'https://www.inform.kz/ru/v-medeuskom-rayone-almaty-sgorel-chastnyy-dom-pogib-pozhiloy-chelovek_a2430361', 'https://www.inform.kz/ru/na-filippinah-ot-pirotehniki-postradali-200-chelovek_a2430344', 'https://www.inform.kz/ru/mezhdunarodnaya-panorama-samye-yarkie-mirovye-novosti-2011-goda_a2430368', 'https://www.inform.kz/ru/prezident-francii-prizval-poddannyh-ne-ustraivat-isteriku_a2430381']...


### Check for pagination links

In [5]:
soup = bs(response.content, 'html.parser')
links = soup.find('p', class_='pagination').find_all('a')
links = set([urlparse.urlparse(URL_MAIN + link['href'].strip()).geturl() for link in links])
links = list(links)
print(f'Pagination links: {links}')

Pagination links: ['https://www.inform.kz/ru/archive/2?date=01.01.2012']


Check for articles from next page:

In [6]:
response = session.get(links[0])
soup = bs(response.content, 'html.parser')

links = soup.find('div', class_='news-list__col').find_all('a')
links = set([urlparse.urlparse(URL_MAIN + link['href'].strip()).geturl() for link in links])
links = list(links)
print(f'Links for articles (total {len(links)} links) from next page:\n{links[:5]}...')

Links for articles (total 2 links) from next page:
['https://www.inform.kz/ru/chislo-zhertv-ciklona-v-indii-vozroslo-do-42_a2430340', 'https://www.inform.kz/ru/ya-ot-vsego-serdca-zhelayu-chtoby-v-novom-godu-sbylis-vse-svetlye-mechty-i-nadezhdy-kazhdogo-kazahstanca-pozdravlenie-prezidenta-rk_a2430289']...


## Check for crawling articles

In [14]:
def check_articles(url):
    response = session.get(url)

    soup = bs(response.content, 'html.parser')

    title = soup.find('div', class_='title_article_bl')
    title = re.sub(r'\s+', ' ', title.get_text().strip())

    date = soup.find('div', class_='time_article_bl')
    date = re.sub(r'\s+', ' ', date.get_text().strip())
    
    links = []
    links_frame = soup.find('div', class_='frame_news_article_adapt')
    if links_frame:
        links = links_frame.find_all('a')
        links = set([urlparse.urlparse(URL_MAIN + link['href'].strip()).geturl() for link in links])
        links = list(links)
        links_frame.decompose()
    
    body = soup.find('div', class_='body_article_bl')
    body = re.sub(r'\s+', ' ', body.get_text().strip())
    
    tags = soup.find('div', class_='keywords_bl')
    if tags:
        tags = tags.find_all('a')
        tags = [re.sub(r'\s+', ' ', t.get_text().strip()) for t in tags]
    else:
        tags = []
    
    author = soup.find('div', class_='data_author_bl')
    if author:
        author = author.find('a')
        author = re.sub(r'\s+', ' ', author.get_text().strip())
    else:
        author = None
    
    print(f'Title: {title}')
    print(f'Date: {date}')
    print(f'Links (number of links - {len(links)}): {links[:3]}')
    print(f'Tags: {tags}')
    print(f'Author: {author}')
    print(f'Body:\n{body}')

In [15]:
url1 = 'https://www.inform.kz/ru/v-zhanaozene-prodolzhaetsya-rabota-po-blagoustroystvu-goroda_a2430452'
url2 = 'https://www.inform.kz/ru/prezident-kazahstana-provel-peregovory-s-general-nym-sekretarem-oon_a3981340'

In [16]:
check_articles(url1)

Title: В Жанаозене продолжается работа по благоустройству города
Date: 2 Января 2012 13:35
Links (number of links - 0): []
Tags: []
Author: Дамир Байманов
Body:
АСТАНА. 2 января. КАЗИНФОРМ - В г. Жанаозен продолжаются мероприятия по вывозу бытового мусора и благоустройству города, сообщает pm.kz со ссылкой на пресс-службу акимата Мангыстауской областиПо информации пресс-службы, работа по очистке города завершена. В то же время продолжается уборка бытового мусора и благоустройство города. По данным на 1 января, в Жанаозене на уборочных работах с ГКП «Тазалык» задействовано 96 человек и 23 единицы специальной техники, вывезено 1154 тонн мусора.


In [17]:
check_articles(url2)

Title: Президент Казахстана провел переговоры с Генеральным секретарем ООН
Date: 21 Сентября 2022 05:06
Links (number of links - 3): ['https://www.inform.kz/ru/glava-gosudarstva-obratilsya-k-narodu-kazahstana_a3981670', 'https://www.inform.kz/ru/glava-gosudarstva-obratilsya-k-narodu-kazahstana_a3981672', 'https://www.inform.kz/ru/glava-gosudarstva-garantiroval-spravedlivye-vybory-v-kazahstane_a3981688']
Tags: ['ООН', 'Президент Казахстана']
Author: Ринат Дусумов
Body:
НЬЮ-ЙОРК. КАЗИНФОРМ - Программа рабочего визита Касым-Жомарта Токаева в Нью-Йорк для участия в 77-й сессии Генеральной Ассамблеи ООН завершилась переговорами с Генеральным секретарем ООН Антониу Гутерришем, передает МИА «Казинформ» со ссылкой на пресс-службу Акорды. В ходе встречи, прошедшей в конструктивной и дружественной атмосфере, стороны обсудили состояние и перспективы укрепления многостороннего сотрудничества между Казахстаном и ООН. Особое внимание было уделено усилиям, направленным на выработку коллективных подхо

## Testing asyncronous HTTP using `aiohttp` package

In [7]:
import aiohttp
import asyncio

In [8]:
async def _get_url(session, url):
    async with session.get(url) as response:
        return await response.text()

In [ ]:
soup = bs(response.content, 'html.parser')
links = soup.find('p', class_='pagination').find_all('a')
links = set([urlparse.urlparse(URL_MAIN + link['href'].strip()).geturl() for link in links])
links = list(links)

In [26]:
async with aiohttp.ClientSession() as session:
    # given the date, get the first page
    url_links = 'https://www.inform.kz/ru/archive?date=01.04.2022'
    response_main = await _get_url(session, url_links)

    # get pagination links
    soup = bs(response_main, 'html.parser')
    links = soup.find('p', class_='pagination').find_all('a')
    links = set([urlparse.urlparse(URL_MAIN + link['href'].strip()).geturl() for link in links])
    links = list(links)

    results = await asyncio.gather(*[
        _get_url(session, link) for link in links
    ])

    all_pages = [response_main] + results
    links_all = []
    for page in all_pages:
        soup = bs(page, 'html.parser')

        links = soup.find('div', class_='news-list__col').find_all('a')
        links = set([urlparse.urlparse(URL_MAIN + link['href'].strip()).geturl() for link in links])
        links_all.extend(list(links))
    print(len(links_all))

    results = await asyncio.gather(*[
        _get_url(session, link) for link in links_all
    ])

140


In [28]:
def retrieve_data(content):
    soup = bs(content, 'html.parser')

    title = soup.find('div', class_='title_article_bl')
    title = re.sub(r'\s+', ' ', title.get_text().strip())

    date = soup.find('div', class_='time_article_bl')
    date = re.sub(r'\s+', ' ', date.get_text().strip())
    
    links = []
    links_frame = soup.find('div', class_='frame_news_article_adapt')
    if links_frame:
        links = links_frame.find_all('a')
        links = set([urlparse.urlparse(URL_MAIN + link['href'].strip()).geturl() for link in links])
        links = list(links)
        links_frame.decompose()
    
    body = soup.find('div', class_='body_article_bl')
    body = re.sub(r'\s+', ' ', body.get_text().strip())
    
    tags = soup.find('div', class_='keywords_bl')
    if tags:
        tags = tags.find_all('a')
        tags = [re.sub(r'\s+', ' ', t.get_text().strip()) for t in tags]
    else:
        tags = []
    
    author = soup.find('div', class_='data_author_bl')
    if author:
        author = author.find('a')
        author = re.sub(r'\s+', ' ', author.get_text().strip())
    else:
        author = None
    
    # print(f'Title: {title}')
    # print(f'Date: {date}')
    # print(f'Links (number of links - {len(links)}): {links[:3]}')
    # print(f'Tags: {tags}')
    # print(f'Author: {author}')
    # print(f'Body:\n{body}')
    
    return {
        'title': title,
        'date': date,
        'body': body,
        'links': links,
        'tags': tags,
        'author': author
    }

In [29]:
res = []
for r in results:
    res.append(retrieve_data(r))

In [30]:
res

[{'title': 'Разъяснения о поправках в закон по противодействию кибербуллингу детей дала депутат Мажилиса',
  'date': '1 Апреля 2022, 22:19',
  'body': 'НУР-СУЛТАН. КАЗИНФОРМ - Депутат Мажилиса Парламента РК Динара Закиева высказалась о нормах касательно блокировки соцсетей, передает МИА «Казинформ». «В настоящее время бурно обсуждаются поправки относящиеся к кибербуллингу в отношении детей и существует мнение о том, что эти поправки могут повлиять на введение цензуры, а также на отключение социальных сетей. В этой связи хотелось бы отметить несколько аспектов: 1) Цель поправок - предусмотреть механизмы оперативной и быстрой защиты детей по фактам кибербуллинга. По данным национального центра общественного здравоохранения 12% детей от 11-15 лет подвергались или участвовали в кибербуллинге. 2) В соответствии с поправками в Законе о СМИ мы предусмотрели порядок рассмотрения обращений граждан и законных представителей ребенка только в отношении кибербуллинга несовершеннолетних. 1. В законо

In [11]:
soup = bs(response, 'html.parser')
links = soup.find('p', class_='pagination').find_all('a')
links = set([urlparse.urlparse(URL_MAIN + link['href'].strip()).geturl() for link in links])
links = list(links)

In [12]:
links

['https://www.inform.kz/ru/archive/2?date=01.01.2012']